# Conditional Parameter Grids

This example shows the usage of `PyExperimenter` with a conditional parameter grid. We will programmatically define the parameter combinations of a support vector machine, instead of generating the entire cartesian product from the parameters defined in the config file.  

To execute this notebook you need to install:
```
pip install py_experimenter
pip install scikit-learn
```

## Experiment Configuration File
This notebook shows an example execution of `PyExperimenter` based on an experiment configuration file. Further explanation about the usage of `PyExperimenter` can be found in the [documentation](https://tornede.github.io/py_experimenter/usage.html). Here, we only define keyfields and resultfields and do not set the parameter values in the experiment configuration file as we will create the parameter grid programmatically.

In [1]:
import os

content = """
[PY_EXPERIMENTER]
provider = sqlite 
database = py_experimenter
table = svm_experiment_example

keyfields = dataset, cross_validation_splits:int, seed:int, kernel, gamma:DECIMAL, degree:int, coef0:DECIMAL

resultfields = train_f1:DECIMAL, train_accuracy:DECIMAL, test_f1:DECIMAL, test_accuracy:DECIMAL
resultfields.timestamps = false

[CUSTOM] 
path = sample_data
"""
experiment_configuration_file_path = os.path.join('config', 'example_conditional_grid.cfg')
with open(experiment_configuration_file_path, "w") as f: 
  f.write(content)

## Defining the execution function

Next, the execution of a single experiment has to be defined. Note that this is a dummy example, which contains limited reasonable code. It is meant to show the core functionality of the PyExperimenter. 

The method is called with the parameters, i.e. `keyfields`, of a database entry. The results are meant to be processed to be written into the database, i.e. as `resultfields`. 

In [2]:
import os
import random

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from py_experimenter.experimenter import PyExperimenter
from py_experimenter.result_processor import ResultProcessor

from time import sleep
from random import randint

def run_svm(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
    sleep(randint(0,5))
    seed = parameters['seed']
    random.seed(seed)
    np.random.seed(seed)

    data = load_iris()

    X = data.data
    y = data.target

    # Create Support Vector Machine with parameters dependent on the kernel
    kernel = parameters['kernel']
    if kernel == 'linear':
        svc = SVC(kernel=parameters['kernel'])
    elif kernel == 'poly':
        svc = SVC(kernel=parameters['kernel'], gamma=parameters['gamma'], coef0=parameters['coef0'], degree=parameters['degree'])
    elif kernel == 'rbf':
        svc = SVC(kernel=parameters['kernel'], gamma=parameters['gamma'])

    svc = SVC()

    model = make_pipeline(StandardScaler(), svc)  

    if parameters['dataset'] != 'iris':
        raise ValueError("Example error")

    scores = cross_validate(model, X, y, 
        cv=parameters['cross_validation_splits'],
        scoring=('accuracy', 'f1_micro'),
        return_train_score=True
    )
    
    result_processor.process_results({
        'train_f1': np.mean(scores['train_f1_micro']),
        'train_accuracy': np.mean(scores['train_accuracy'])
    })

    result_processor.process_results({
        'test_f1': np.mean(scores['test_f1_micro']),
        'test_accuracy': np.mean(scores['test_accuracy'])})

## Executing PyExperimenter

The actual execution of the PyExperimenter is done in multiple steps. 

### Initialize PyExperimenter
The PyExperimenter is initialized with the previously created configuration file. Additionally, `PyExperimenter` is given a `name`, i.e. job id, which is especially useful for parallel executions of multiple experiments on HPC. 

In [3]:
from py_experimenter.experimenter import PyExperimenter

experimenter = PyExperimenter(experiment_configuration_file_path=experiment_configuration_file_path, name="SVM_experimenter_01")

### Fill Table

The table is filled programmatically using the  `fill_table_from_combination()` method. We first generate the fixed parameter combinations for each kernel of the SVM in the first three lines.
* For the `rbf` kernel, we need to set values for the `gamma` parameter. The degree and `coef0` parameter are not present in this kernel, so we set these to `'nan'`.
* For the `poly` kernel, we need to set the `gamma`, the degree as well as the `coef0` parameter.
* For the `linear` kernel, we do not need to set any parameters, so all of them are set to `'nan'`.

Afterwards, we combine these with the seed, the dataset and the cross_validation_splits parameters, which are present for all experiment runs. Thus, these are not set unconditionally.

 

Note that the table can easily be obtained as `pandas.Dataframe` via `experimenter.get_table()`.

In [4]:
# Create parameter configurations for each kernel
combinations = [{'kernel': 'rbf', 'gamma': gamma, 'degree':'nan', 'coef0':'nan'} for gamma in ['0.1','0.3']]
combinations += [{'kernel': 'poly', 'gamma': gamma, 'degree': degree, 'coef0': coef0} for gamma in ['0.1','0.3'] for degree in ['3','4'] for coef0 in ['0.0', '0.1']]
combinations += [{'kernel': 'linear','gamma': 'nan', 'degree':'nan', 'coef0':'nan'}]

# Fill experimenter
experimenter.fill_table_from_combination(parameters={'seed': ['1', '2', '3', '4', '5'], 
'dataset': ['iris'],
'cross_validation_splits': ['5'] },
fixed_parameter_combinations=combinations)

# showing database table
experimenter.get_table()

,ID,dataset,cross_validation_splits,seed,kernel,gamma,degree,coef0,creation_date,status,start_date,name,machine,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,1,rbf,0.1,nan,nan,2023-03-28 16:34:55,done,2023-03-31 16:19:27,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:19:28,None
1,2,iris,5,1,rbf,0.3,nan,nan,2023-03-28 16:34:55,done,2023-03-31 16:21:01,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:21:02,None
2,3,iris,5,1,poly,0.1,3,0,2023-03-28 16:34:55,done,2023-03-31 16:20:50,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:20:51,None
3,4,iris,5,1,poly,0.1,3,0.1,2023-03-28 16:34:55,done,2023-03-31 16:20:31,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:20:35,None
4,5,iris,5,1,poly,0.1,4,0,2023-03-28 16:34:55,done,2023-03-31 16:20:03,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:20:04,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,111,iris,5,4,poly,0.3,4,0,2023-03-31 16:33:28,created,None,None,None,NaN,NaN,NaN,NaN,None,None
111,112,iris,5,5,poly,0.1,3,0,2023-03-31 16:33:28,created,None,None,None,NaN,NaN,NaN,NaN,None,None
112,113,iris,5,5,poly,0.1,4,0,2023-03-31 16:33:28,created,None,None,None,NaN,NaN,NaN,NaN,None,None
113,114,iris,5,5,poly,0.3,3,0,2023-03-31 16:33:28,created,None,None,None,NaN,NaN,NaN,NaN,None,None


### Execute PyExperimenter
All experiments are executed one after the other by the same `PyExperimenter` due to `max_experiments=-1`. If just a single one or a predifined number of experiments should be executed, the `-1` has to be replaced by the according amount. The `random_order` is especially important in case of parallel execution of multiple `PyExperimenter`, e.g. when doing it on a HPC, to avoid collusions of accessing the same row of the table. 

The first parameter, i.e. `run_svm`, relates to the actual method that should be executed with the given keyfields of the table. 

In [5]:
experimenter.execute(run_svm, max_experiments=-1, random_order=True)

# showing database table
experimenter.get_table() 

,ID,dataset,cross_validation_splits,seed,kernel,gamma,degree,coef0,creation_date,status,start_date,name,machine,train_f1,train_accuracy,test_f1,test_accuracy,end_date,error
0,1,iris,5,1,rbf,0.1,nan,nan,2023-03-28 16:34:55,done,2023-03-31 16:19:27,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:19:28,None
1,2,iris,5,1,rbf,0.3,nan,nan,2023-03-28 16:34:55,done,2023-03-31 16:21:01,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:21:02,None
2,3,iris,5,1,poly,0.1,3,0,2023-03-28 16:34:55,done,2023-03-31 16:20:50,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:20:51,None
3,4,iris,5,1,poly,0.1,3,0.1,2023-03-28 16:34:55,done,2023-03-31 16:20:31,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:20:35,None
4,5,iris,5,1,poly,0.1,4,0,2023-03-28 16:34:55,done,2023-03-31 16:20:03,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:20:04,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,111,iris,5,4,poly,0.3,4,0,2023-03-31 16:33:28,done,2023-03-31 16:33:48,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:33:49,None
111,112,iris,5,5,poly,0.1,3,0,2023-03-31 16:33:28,done,2023-03-31 16:33:39,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:33:40,None
112,113,iris,5,5,poly,0.1,4,0,2023-03-31 16:33:28,done,2023-03-31 16:33:50,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:33:51,None
113,114,iris,5,5,poly,0.3,3,0,2023-03-31 16:33:28,done,2023-03-31 16:33:28,SVM_experimenter_01,MacBook-Pro-von-Tanja.fritz.box,0.975,0.975,0.966667,0.966667,2023-03-31 16:33:30,None
